In [2]:
import pickle

X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
X_t = pickle.load(open('X_t.pickle','rb'))
y_t = pickle.load(open('y_t.pickle','rb'))

X = X/255.0
X_t = X_t/255.0

In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB2 ,EfficientNetB5
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
IMAGE_SIZE = [48, 48]

# add preprocessing layer to the front of VGG

pretrained_model = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in pretrained_model.layers:
    layer.trainable = False
        
pretrained_model.layers[0].trainable = True
pretrained_model.layers[1].trainable = True
pretrained_model.layers[2].trainable = True     
pretrained_model.layers[3].trainable = True 
pretrained_model.layers[4].trainable = True 
pretrained_model.layers[5].trainable = True 
pretrained_model.layers[-1].trainable = True
pretrained_model.layers[-2].trainable = True
pretrained_model.layers[-3].trainable = True

In [5]:


# our layers - you can add more if you want
x = Flatten()(pretrained_model.output)
x2 = Dense(256, activation='relu')(x)
x3 = Dropout(0.2)(x2)
prediction = Dense(5, activation='softmax')(x3)

# create a model object
model = Model(inputs=pretrained_model.input, outputs=prediction)


# view the structure of the model
model.summary()

optimizer = Adam(lr = 0.001)

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 48, 48, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 48, 48, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 49, 49, 3)    0           normalization[0][0]              
______________________________________________________________________________________________

In [6]:
model.fit(X, y,validation_data=(X_t, y_t), batch_size=32, epochs=10)

Epoch 1/10
1569/1569 [==============================] - 132s 74ms/step - loss: 1.6348 - accuracy: 0.3018 - val_loss: 1.6583 - val_accuracy: 0.2189
Epoch 2/10
1569/1569 [==============================] - 109s 69ms/step - loss: 1.4217 - accuracy: 0.4102 - val_loss: 1.6645 - val_accuracy: 0.1535
Epoch 3/10
1569/1569 [==============================] - 109s 70ms/step - loss: 1.3672 - accuracy: 0.4416 - val_loss: 1.7019 - val_accuracy: 0.2850
Epoch 4/10
1569/1569 [==============================] - 109s 70ms/step - loss: 1.3246 - accuracy: 0.4667 - val_loss: 2.0270 - val_accuracy: 0.1999
Epoch 5/10
1569/1569 [==============================] - 109s 70ms/step - loss: 1.2913 - accuracy: 0.4828 - val_loss: 1.6267 - val_accuracy: 0.2345
Epoch 6/10
1569/1569 [==============================] - 86s 55ms/step - loss: 1.2586 - accuracy: 0.4941 - val_loss: 1.6370 - val_accuracy: 0.2400
Epoch 7/10
1569/1569 [==============================] - 75s 48ms/step - loss: 1.2262 - accuracy: 0.5117 - val_loss: 1.6

In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_10epochs.h5")
print("Saved model to disk")

Saved model to disk


In [8]:
optimizer = Adam(lr = 0.0003)

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)

model.fit(X, y,validation_data=(X_t, y_t), batch_size=32, epochs=10)

Epoch 1/10
1569/1569 [==============================] - 121s 70ms/step - loss: 1.0916 - accuracy: 0.5659 - val_loss: 1.3579 - val_accuracy: 0.4508
Epoch 2/10
1569/1569 [==============================] - 107s 68ms/step - loss: 1.0450 - accuracy: 0.5861 - val_loss: 1.5932 - val_accuracy: 0.3483
Epoch 3/10
1569/1569 [==============================] - 107s 68ms/step - loss: 1.0264 - accuracy: 0.5932 - val_loss: 2.7227 - val_accuracy: 0.2846
Epoch 4/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.9998 - accuracy: 0.6031 - val_loss: 1.9244 - val_accuracy: 0.2908
Epoch 5/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.9764 - accuracy: 0.6141 - val_loss: 1.9989 - val_accuracy: 0.2975
Epoch 6/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.9599 - accuracy: 0.6215 - val_loss: 4.0771 - val_accuracy: 0.2705
Epoch 7/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.9489 - accuracy: 0.6283 - val_loss: 1

In [9]:

model.save_weights("model_20epochs.h5")
print("Saved model to disk")

Saved model to disk


In [11]:
optimizer = Adam(lr = 0.00001)

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)

model.fit(X, y,validation_data=(X_t, y_t), batch_size=32, epochs=10)

Epoch 1/10
1569/1569 [==============================] - 120s 70ms/step - loss: 0.8192 - accuracy: 0.6771 - val_loss: 1.3054 - val_accuracy: 0.5412
Epoch 2/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.8113 - accuracy: 0.6846 - val_loss: 1.3017 - val_accuracy: 0.5449
Epoch 3/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.8076 - accuracy: 0.6860 - val_loss: 1.3148 - val_accuracy: 0.5366
Epoch 4/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.7961 - accuracy: 0.6892 - val_loss: 1.3412 - val_accuracy: 0.5398
Epoch 5/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.8006 - accuracy: 0.6878 - val_loss: 1.3124 - val_accuracy: 0.5431
Epoch 6/10
1569/1569 [==============================] - 107s 68ms/step - loss: 0.8037 - accuracy: 0.6880 - val_loss: 1.3182 - val_accuracy: 0.5398
Epoch 7/10
1569/1569 [==============================] - 108s 69ms/step - loss: 0.8092 - accuracy: 0.6837 - val_loss: 1

In [12]:

model.save_weights("model_30epochs.h5")
print("Saved model to disk")

Saved model to disk
